In [19]:
from pina.problem import SpatialProblem
from pina import Condition,LabelTensor,Trainer
from pina.model import FeedForward
from pina.solvers import PINN
from pina.geometry import Difference, CartesianDomain, EllipsoidDomain
import matplotlib.pyplot as plt

class Stokes(SpatialProblem):
    outputvariables = ['u']
    spatial_domain = Difference([CartesianDomain({'x':[0, 2.2], 'y':[0, 0.41]}), EllipsoidDomain({'x':[-0.5, 0.5], 'y':[-0.5, 0.5]})])

    def laplace_equation(output_,input_):
        return - 0.05 * laplacian(output_,input_,components=['ux','uy'],d=['x','y']) + div(output_,input_,components=['P'],d=['x','y'])

    def div_equation(output_,input_):
        return div(output_,input_,components=['ux','uy'],d=['x','y'])

    list_eq = [laplace_equation,div_equation]
    stokes_eq = SystemEquation[list_eq,reduction='mean']
    def partials_equation(output_,input_):
        x_eq = 0.05 * grad(output_,input_,components=['ux','uy'],d=['x'] - P)
        y_eq = 0.05 * grad(output_,input_,components=['ux','uy'],d=['y'])
        list_parti_eq = [x_eq,y_eq]
        return SystemEquation(list_parti_eq,reduction='mean')

    conditions = {
        'gamma1': Condition(location=CartesianDomain({'x':[1, 5], 'y': 3}), equation=FixedValue(0.)),
        'gamma2': Condition(location=CartesianDomain({'x':[1, 5], 'y': 1}), equation=FixedValue(0.)),
        'gamma3': Condition(location=CartesianDomain({'x': 1, 'y':[1, 3]), equation=FixedValue(0.)), # fixed value u_in
        'gamma4': Condition(location=CartesianDomain({'x': 5, 'y':[1, 3]), equation=FixedValue(0.)),
        'D': Condition(location=CartesianDomain({'x':[1, 5], 'y': 3}), equation=Equation(stokes_eq)),